In [1]:
import pandas as pd
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import model_selection

from sklearn.datasets import fetch_20newsgroups
from keras.layers import  Dropout, Dense
from keras.models import Sequential

Using TensorFlow backend.


In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
id='1Pk5MK9Hs_kMUT9NotGnOKE0NPra-39YU'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('train.csv')

In [4]:
train = pd.read_csv('train.csv')

In [5]:
id='1GsTM9oLtIV8-Da_fDOFWsQYMpgQ8GOYJ'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('test.csv')

In [6]:
test = pd.read_csv('test.csv')

In [7]:
id='1ttcOEkPasTJe2OHU-NtcJa9IrGZT7hVM'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('features_test_doc2vec.csv')

In [8]:
features_test = pd.read_csv('features_test_doc2vec.csv')

In [9]:
features_test.shape

(3263, 108)

In [10]:
id='16bCZV4op8UrlgwJEc2KS4cQEaLEYMMTv'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('features_train_doc2vec.csv')

In [11]:
features_train = pd.read_csv('features_train_doc2vec.csv')

In [12]:
features_train.shape

(7613, 108)

In [13]:
y = train.target
x_train, x_validation, y_train, y_validation = train_test_split(features_train, y, test_size=0.3, stratify=y)

##KNN

Buscamos los mejores hiperparametros

In [16]:
#BUSQUEDA DE HIPERPARAMETROS
knn = KNeighborsClassifier()
params_knn = {'n_neighbors': np.arange(1, 25)}
knn_gs = GridSearchCV(knn, params_knn, cv=5)
knn_gs.fit(x_train, y_train)

knn_best = knn_gs.best_estimator_
print(knn_gs.best_params_)

print('knn score: {}'.format(knn_best.score(x_validation, y_validation)))

{'n_neighbors': 8}
knn score: 0.6711908931698775


Realizamos la prediccion con el modelo

In [ ]:
prediction_knn = knn_best.predict(features_test)
submission_knn = pd.DataFrame(data={'id':test['id'], 'target': prediction_knn})
submission_knn.to_csv('doc2vec_knn.csv', index=False)

#RED NEURONAL PROFUNDA

In [19]:
def DNN_model(shape, nClasses, dropout=0.5):
    model = Sequential()
    node = 512
    nLayers = 4
    model.add(Dense(node,input_dim=shape,activation='relu'))
    model.add(Dropout(dropout))
    for i in range(0,nLayers):
        model.add(Dense(node,input_dim=node,activation='relu'))
        model.add(Dropout(dropout))
    model.add(Dense(nClasses, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [21]:
model_DNN = DNN_model(features_train.shape[1], 2)
model_DNN.fit(features_train, train.target, epochs=20, batch_size=128, verbose=2)

Epoch 1/20
 - 2s - loss: 1.8808 - accuracy: 0.5136
Epoch 2/20
 - 2s - loss: 0.7776 - accuracy: 0.5292
Epoch 3/20
 - 2s - loss: 0.7242 - accuracy: 0.5328
Epoch 4/20
 - 2s - loss: 0.7075 - accuracy: 0.5431
Epoch 5/20
 - 2s - loss: 0.6998 - accuracy: 0.5455
Epoch 6/20
 - 2s - loss: 0.6954 - accuracy: 0.5479
Epoch 7/20
 - 2s - loss: 0.6894 - accuracy: 0.5635
Epoch 8/20
 - 2s - loss: 0.6930 - accuracy: 0.5589
Epoch 9/20
 - 2s - loss: 0.6887 - accuracy: 0.5622
Epoch 10/20
 - 2s - loss: 0.6880 - accuracy: 0.5623
Epoch 11/20
 - 2s - loss: 0.6876 - accuracy: 0.5605
Epoch 12/20
 - 2s - loss: 0.6850 - accuracy: 0.5638
Epoch 13/20
 - 2s - loss: 0.6846 - accuracy: 0.5636
Epoch 14/20
 - 2s - loss: 0.6826 - accuracy: 0.5652
Epoch 15/20
 - 2s - loss: 0.6830 - accuracy: 0.5657
Epoch 16/20
 - 2s - loss: 0.6783 - accuracy: 0.5689
Epoch 17/20
 - 2s - loss: 0.6715 - accuracy: 0.5749
Epoch 18/20
 - 2s - loss: 0.6650 - accuracy: 0.5843
Epoch 19/20
 - 2s - loss: 0.6616 - accuracy: 0.5870
Epoch 20/20
 - 2s - l

In [32]:
prediction_dnn = model_DNN.predict_classes(features_test)

In [ ]:
submission_dnn = pd.DataFrame(data={'id':test['id'], 'target': prediction_dnn})
submission_dnn.to_csv('doc2vec_dnn.csv', index=False)

#RANDOM FOREST

GRID SEARCH

In [22]:
rf = RandomForestClassifier()
params_rf = {'n_estimators': [50, 100, 200]}
rf_gs = GridSearchCV(rf, params_rf, cv=5)
rf_gs.fit(x_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [23]:
rf_best = rf_gs.best_estimator_
print(rf_gs.best_params_)

{'n_estimators': 200}


In [24]:
print('rf: {}'.format(rf_best.score(x_validation, y_validation)))

rf: 0.6637478108581436


In [ ]:
prediction_rf = rf_best.predict(features_test)

In [ ]:
submission_rf = pd.DataFrame(data={'id':test['id'], 'target': prediction_rf})
submission_rf.to_csv('doc2vec_rf.csv', index=False)

BAGGING

In [25]:
seed = 7
num_trees = 100

BAGGING CLASSIFIER

In [26]:
kfold = model_selection.KFold(n_splits=10)
cart = DecisionTreeClassifier()
model = BaggingClassifier(base_estimator=cart, n_estimators=num_trees, random_state=seed)
results = model_selection.cross_val_score(model, features_train, train.target, cv=kfold)
print(results.mean())

0.6308928368185251


In [ ]:
prediction_bagg = model.predict(features_test)

In [ ]:
#submission_bagg = pd.DataFrame(data={'id':test['id'], 'target': prediction_bagg})
#submission_bagg.to_csv('doc2vec_bagg.csv', index=False)

BAGGING CON RANDOM FOREST

In [27]:
max_features = 3
kfold = model_selection.KFold(n_splits=10)
model = RandomForestClassifier(n_estimators=num_trees, max_features=max_features)
results = model_selection.cross_val_score(model, features_train, train.target, cv=kfold)
print(results.mean())

0.6152563797462242


In [ ]:
prediction_rf_bagg = model.predict(features_test)

In [ ]:
submission_rf_bagg = pd.DataFrame(data={'id':test['id'], 'target': prediction_rf_bagg})
submission_rf_bagg.to_csv('doc2vec_rf_bagg.csv', index=False)

#NuSCV

TARDO NADA pero mala prediccion. MUY BUENA PREDICCION SOLO CON TFIDF SIN MAS FEATURES.

In [28]:
nuscv = NuSVC(probability=True)
nuscv.fit(x_train, y_train)

print('rf: {}'.format(nuscv.score(x_validation, y_validation)))

rf: 0.5901926444833625


In [ ]:
prediction_nuscv = nuscv.predict(features_test)

submission_nuscv = pd.DataFrame(data={'id':test['id'], 'target': prediction_nuscv})
submission_nuscv.to_csv('doc2vec_nuscv.csv', index=False)

#XGBoost

In [30]:
model = XGBClassifier()
model.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [31]:
y_pred = model.predict(x_validation)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_validation, predictions)
print("Accuracy: %.5f%%" % (accuracy * 100.0))

Accuracy: 67.77583%


In [ ]:
submission_xgboost = pd.DataFrame(data={'id':test['id'], 'target': prediction_xgboost})
submission_xgboost.to_csv('tf_idf_xgboost.csv', index=False)